<a href="https://colab.research.google.com/github/LIMWAER/Bus-schedule-API/blob/master/Bus_schedule_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://mybuses.ru/moscow/"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
div = soup.find("div", class_="list-group")
div

<div class="list-group"> <a href="/moscow/bus/1/" title="Расписание автобуса №1 - МГУ (к/ст) - Севастопольский просп. (к/ст, выс.)"><b>1</b> - МГУ (к/ст) - Севастопольский просп. (к/ст, выс.)</a> <a href="/moscow/bus/2/" title="Расписание автобуса №2 - 4-й мкр. Митина (к/ст) - Метро `Тушинская` (к/ст)"><b>2</b> - 4-й мкр. Митина (к/ст) - Метро "Тушинская" (к/ст)</a> <a href="/moscow/bus/3/" title="Расписание автобуса №3 - Камчатская ул. (к/ст) - Метро `Бульв. Рокоссовского`"><b>3</b> - Камчатская ул. (к/ст) - Метро "Бульв. Рокоссовского"</a> <a href="/moscow/bus/4/" title="Расписание автобуса №4 - Силикатный з-д (к/ст) - Метро `Краснопресненская` (выс.)"><b>4</b> - Силикатный з-д (к/ст) - Метро "Краснопресненская" (выс.)</a> <a href="/moscow/bus/5/" title="Расписание автобуса №5 - 9-й мкр. Солнцева (к/ст) - Платф. Переделкино (выс.)"><b>5</b> - 9-й мкр. Солнцева (к/ст) - Платф. Переделкино (выс.)</a> <a href="/moscow/bus/7/" title="Расписание автобуса №7 - Ст. Перово (к/ст) - Метро `Па

# Прототип функции парсинга таблиц:

In [4]:
def parse_schedule(a):
  url = "https://mybuses.ru"+a["href"]
  page = requests.get(url)
  soup = BeautifulSoup(page.text, "html.parser")
  dfs = []

  def get_rows(table):
    rows = []
    for tr in table.find_all('tr'):
      row = [td.string for td in tr.find_all('td')]
      if len(row) == 0:
        continue
      rows.append(row)
    return rows


  def get_cols(table):
    cols = []  
    for th in table.find_all('th'):
      try:
        cols.append(th.a.string)
      except AttributeError:
        cols.append(th.string)
    return cols

  for table in soup.find_all('table'):
    df = pd.DataFrame(data = get_rows(table), columns=get_cols(table))
    dfs.append(df)
  #print(dfs) 
  return dfs


In [7]:
%%time
for a in div.find_all('a'):
  dfs = parse_schedule(a)
  break

CPU times: user 272 ms, sys: 3.07 ms, total: 275 ms
Wall time: 680 ms


In [8]:
dfs

[   График / Остановка  ... Севастопольский просп. (к/ст, выс.)
 0            выходные  ...                               06:08
 1               будни  ...                               06:27
 2            выходные  ...                               06:41
 3               будни  ...                               06:47
 4               будни  ...                               07:09
 ..                ...  ...                                 ...
 71              будни  ...                               23:49
 72           выходные  ...                               00:33
 73              будни  ...                               00:41
 74           выходные  ...                               01:20
 75              будни  ...                               01:33
 
 [76 rows x 38 columns],
    График / Остановка Севастопольский просп. (к/ст, выс.)  ... МГУ (выс.) МГУ (к/ст)
 0               будни                               05:29  ...      06:04      06:05
 1            выходные           